# Solving Maze with Q-learning  ( Reinforcement Learning)

Maze generator consists of maze class and field class. It generates a square shaped maze. The maze has route tiles, wall and block tiles, starting and goal point. The route tiles have -1 or 0 on it, which is the point you can get by stepping it. Apparently you will get 1 point subtracted if you step on -1 tile. The wall and block tiles, in #, are where you cannot interude. You have to bypass #. The starting point, namely S, is where you start the maze and goal point, which is shown as 50, is where you head to. You will earn 50 points when you made to the goal.

### Objective of this notebook is to solve self-made maze with Q-learning.
### The maze is in square shape, consists of start point, goal point and tiles in the mid of them.
### Each tile has numericals as its point. In other words, if you step on to the tile with -1, you get 1 point subtracted.
### The maze has blocks to prevent you from taking the route.

In [1]:
import numpy as np
import pandas as pds
import random
import copy
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from collections import deque
from keras import backend as K

# Maze Class

In [9]:
class Maze(object):
    def __init__(self, size=10, blocks_rate=0.1):
        self.size = size if size > 3 else 10
        self.blocks = int((size ** 2) * blocks_rate) 
        self.s_list = []
        self.maze_list = []
        self.e_list = []

    def create_mid_lines(self, k):
        if k == 0: self.maze_list.append(self.s_list)
        elif k == self.size - 1: self.maze_list.append(self.e_list)
        else:
            tmp_list = []
            for l in range(0,self.size):
                if l == 0: tmp_list.extend("#")
                elif l == self.size-1: tmp_list.extend("#")
                else:
                    a = random.randint(-1, 0)
                    tmp_list.extend([a])
            self.maze_list.append(tmp_list)

    def insert_blocks(self, k, s_r, e_r):
        b_y = random.randint(1, self.size-2)
        b_x = random.randint(1, self.size-2)
        if [b_y, b_x] == [1, s_r] or [b_y, b_x] == [self.size - 2, e_r]: k = k-1
        else: self.maze_list[b_y][b_x] = "#"
            
    def generate_maze(self): 
        s_r = random.randint(1, (self.size / 2) - 1)
        for i in range(0, self.size):
            if i == s_r: self.s_list.extend("S")
            else: self.s_list.extend("#")
        start_point = [0, s_r]

        e_r = random.randint((self.size / 2) + 1, self.size - 2)
        for j in range(0, self.size):
            if j == e_r: self.e_list.extend([50])
            else: self.e_list.extend("#")
        goal_point = [self.size - 1, e_r]

        for k in range(0, self.size):
            self.create_mid_lines(k)
        
        for k in range(self.blocks):
            self.insert_blocks(k, s_r, e_r)

        return self.maze_list, start_point, goal_point

# Maze functions

In [10]:
class Field(object):
    def __init__(self, maze, start_point, goal_point):
        self.maze = maze
        self.start_point = start_point
        self.goal_point = goal_point
        self.movable_vec = [[1,0],[-1,0],[0,1],[0,-1]]

    def display(self, point=None):
        field_data = copy.deepcopy(self.maze)
        if not point is None:
                y, x = point
                field_data[y][x] = "@@"
        else:
                point = ""
        for line in field_data:
                print ("\t" + "%3s " * len(line) % tuple(line))

    def get_actions(self, state):
        movables = []
        if state == self.start_point:
            y = state[0] + 1
            x = state[1]
            a = [[y, x]]
            return a
        else:
            for v in self.movable_vec:
                y = state[0] + v[0]
                x = state[1] + v[1]
                if not(0 < x < len(self.maze) and
                       0 <= y <= len(self.maze) - 1 and
                       maze[y][x] != "#" and
                       maze[y][x] != "S"):
                    continue
                movables.append([y,x])
            if len(movables) != 0:
                return movables
            else:
                return None

    def get_val(self, state):
        y, x = state
        if state == self.start_point: return 0, False
        else:
            v = float(self.maze[y][x])
            if state == self.goal_point: 
                return v, True
            else: 
                return v, False

# Generate a maze

In [11]:
size = 10
barriar_rate = 0.1

In [12]:
maze_1 = Maze(size, barriar_rate)
maze, start_point, goal_point = maze_1.generate_maze()
maze_field = Field(maze, start_point, goal_point)

In [13]:
maze_field.display()

	  #   S   #   #   #   #   #   #   #   # 
	  #  -1   #   0   0  -1  -1   0   0   # 
	  #   0  -1   0  -1   0   0  -1   0   # 
	  #   0   0   0   0  -1  -1   #   0   # 
	  #  -1   #  -1   0  -1  -1   0  -1   # 
	  #  -1   0   0  -1   0   0  -1  -1   # 
	  #  -1   #   0  -1  -1   0  -1  -1   # 
	  #   #   #   0   0   0   #  -1   0   # 
	  #  -1  -1  -1  -1   0  -1   0   0   # 
	  #   #   #   #   #   #   #  50   #   # 


# Solving the maze in Q-learning

In [14]:
class QLearning_Solver(object):
    def __init__(self, maze, display=False):
        self.Qvalue = {}
        self.Field = maze
        self.alpha = 0.2
        self.gamma  = 0.9
        self.epsilon = 0.2
        self.steps = 0
        self.score = 0
        self.display = display

    def qlearn(self, greedy_flg=False):
        state = self.Field.start_point
        while True:
            if greedy_flg:
                self.steps += 1
                action = self.choose_action_greedy(state)
                print("current state: {0} -> action: {1} ".format(state, action))
                if self.display:
                    self.Field.display(action)
                reward, tf = self.Field.get_val(action)
                self.score =  self.score + reward
                print("current step: {0} \t score: {1}\n".format(self.steps, self.score))
                if tf == True:
                    print("Goal!")
                    break
            else:
                action = self.choose_action(state)    
            if self.update_Qvalue(state, action):
                break
            else:
                state = action

    def update_Qvalue(self, state, action):
        Q_s_a = self.get_Qvalue(state, action)
        mQ_s_a = max([self.get_Qvalue(action, n_action) for n_action in self.Field.get_actions(action)])
        r_s_a, finish_flg = self.Field.get_val(action)
        q_value = Q_s_a + self.alpha * ( r_s_a +  self.gamma * mQ_s_a - Q_s_a)
        self.set_Qvalue(state, action, q_value)
        return finish_flg


    def get_Qvalue(self, state, action):
        state = (state[0],state[1])
        action = (action[0],action[1])
        try:
            return self.Qvalue[state][action]
        except KeyError:
            return 0.0

    def set_Qvalue(self, state, action, q_value):
        state = (state[0],state[1])
        action = (action[0],action[1])
        self.Qvalue.setdefault(state,{})
        self.Qvalue[state][action] = q_value

    def choose_action(self, state):
        if self.epsilon < random.random():
            return random.choice(self.Field.get_actions(state))
        else:
            return self.choose_action_greedy(state)

    def choose_action_greedy(self, state):
        best_actions = []
        max_q_value = -100
        for a in self.Field.get_actions(state):
            q_value = self.get_Qvalue(state, a)
            if q_value > max_q_value:
                best_actions = [a,]
                max_q_value = q_value
            elif q_value == max_q_value:
                best_actions.append(a)
        return random.choice(best_actions)

    def dump_Qvalue(self):
        print("##### Dump Qvalue #####")
        for i, s in enumerate(self.Qvalue.keys()):
            for a in self.Qvalue[s].keys():
                print("\t\tQ(s, a): Q(%s, %s): %s" % (str(s), str(a), str(self.Qvalue[s][a])))
            if i != len(self.Qvalue.keys())-1: 
                print('\t------------------state   action   reward')

In [15]:
learning_count = 1000

In [16]:
QL_solver = QLearning_Solver(maze_field, display=True)

In [17]:
for i in range(learning_count):
    QL_solver.qlearn()

In [18]:
QL_solver.dump_Qvalue()

##### Dump Qvalue #####
		Q(s, a): Q((0, 1), (1, 1)): 9.56547669099945
	------------------state   action   reward
		Q(s, a): Q((1, 1), (2, 1)): 11.739418545554948
	------------------state   action   reward
		Q(s, a): Q((2, 1), (2, 2)): 12.043798383949946
		Q(s, a): Q((2, 1), (3, 1)): 13.043798383949946
		Q(s, a): Q((2, 1), (1, 1)): 9.56547669099945
	------------------state   action   reward
		Q(s, a): Q((2, 2), (2, 1)): 11.739418545554948
		Q(s, a): Q((2, 2), (2, 3)): 14.493109315499943
		Q(s, a): Q((2, 2), (3, 2)): 14.493109315499943
	------------------state   action   reward
		Q(s, a): Q((2, 3), (2, 2)): 12.043798383949946
		Q(s, a): Q((2, 3), (2, 4)): 14.796157894999949
		Q(s, a): Q((2, 3), (1, 3)): 13.043798383949946
		Q(s, a): Q((2, 3), (3, 3)): 16.10345479499994
	------------------state   action   reward
		Q(s, a): Q((2, 4), (1, 4)): 13.316542105499202
		Q(s, a): Q((2, 4), (3, 4)): 17.551286549999947
		Q(s, a): Q((2, 4), (2, 3)): 14.493109315499943
		Q(s, a): Q((2, 4), (2, 5)): 1

In [19]:
QL_solver.qlearn(greedy_flg=True)

current state: [0, 1] -> action: [1, 1] 
	  #   S   #   #   #   #   #   #   #   # 
	  #  @@   #   0   0  -1  -1   0   0   # 
	  #   0  -1   0  -1   0   0  -1   0   # 
	  #   0   0   0   0  -1  -1   #   0   # 
	  #  -1   #  -1   0  -1  -1   0  -1   # 
	  #  -1   0   0  -1   0   0  -1  -1   # 
	  #  -1   #   0  -1  -1   0  -1  -1   # 
	  #   #   #   0   0   0   #  -1   0   # 
	  #  -1  -1  -1  -1   0  -1   0   0   # 
	  #   #   #   #   #   #   #  50   #   # 
current step: 1 	 score: -1.0

current state: [1, 1] -> action: [2, 1] 
	  #   S   #   #   #   #   #   #   #   # 
	  #  -1   #   0   0  -1  -1   0   0   # 
	  #  @@  -1   0  -1   0   0  -1   0   # 
	  #   0   0   0   0  -1  -1   #   0   # 
	  #  -1   #  -1   0  -1  -1   0  -1   # 
	  #  -1   0   0  -1   0   0  -1  -1   # 
	  #  -1   #   0  -1  -1   0  -1  -1   # 
	  #   #   #   0   0   0   #  -1   0   # 
	  #  -1  -1  -1  -1   0  -1   0   0   # 
	  #   #   #   #   #   #   #  50   #   # 
current step: 2 	 score: -1.0

current state: [